# AI cập nhật tin tức mới nhất cho báo cáo tuần

In [1]:
import sys
import os
import importlib
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'import'))
sys.path.append(os.path.join(os.path.dirname(os.getcwd()), 'module'))

import import_default
import import_database
import import_other
import get_and_crawl_data
import gemini_model
import plotly_and_upload

importlib.reload(import_default)
importlib.reload(import_database)
importlib.reload(import_other)
importlib.reload(get_and_crawl_data)
importlib.reload(gemini_model)
importlib.reload(plotly_and_upload)

from import_default import *
from import_database import *
from import_other import *
from get_and_crawl_data import *
from gemini_model import *
from plotly_and_upload import *

In [2]:
# Thiết lập kết nối với cơ sở dữ liệu
genai.configure(api_key=load_env("GEMINI_API"))
model_list = get_gemini_models()

# Sắp xếp danh sách model theo thứ tự ổn định
fast_model_list = select_fast_models(model_list)
standard_model_list = select_standard_models(model_list)

# Tạo dictionary cho các model
fast_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in fast_model_list}
standard_model_dict = {model_name: genai.GenerativeModel(model_name) for model_name in standard_model_list}

In [3]:
vietstock_rss_url_dict = {
    'trong_nuoc': {
        'vi_mo': 'https://vietstock.vn/761/kinh-te/vi-mo.rss',
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 'https://vietstock.vn/772/the-gioi/tai-chinh-quoc-te.rss',
        'chung_khoan_the_gioi': 'https://vietstock.vn/773/the-gioi/chung-khoan-the-gioi.rss',
    },
    'doanh_nghiep': {
        'hoat_dong_kinh_doanh': 'https://vietstock.vn/737/doanh-nghiep/hoat-dong-kinh-doanh.rss',
    }
}

cafef_rss_url_dict = {
    'trong_nuoc': {
        'kinh_te_vi_mo': 'https://cafef.vn/vi-mo-dau-tu.chn',
        'chung_khoan': 'https://cafef.vn/thi-truong-chung-khoan.chn',
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 'https://cafef.vn/tai-chinh-quoc-te.chn',
    },
    'doanh_nghiep': {
        'doanh_nghiep': 'https://cafef.vn/doanh-nghiep.chn',
    }
}

vietnambiz_rss_url_dict = {
    'trong_nuoc': {
        'chung_khoan': 'https://vietnambiz.vn/chung-khoan/thi-truong.htm',
        'nha_dat': 'https://vietnambiz.vn/nha-dat/thi-truong.htm',
    },
    'quoc_te': {
        'tai_chinh_chung_khoan': 'https://vietnambiz.vn/quoc-te/tai-chinh-chung-khoan.htm',
    },
    'doanh_nghiep': {
        'ket_qua_kinh_doanh': 'https://vietnambiz.vn/doanh-nghiep/ket-qua-kinh-doanh.htm',
    }
}

In [4]:
vietstock_count_dict = {
    'trong_nuoc': {
        'vi_mo': 4,
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 4,
        'chung_khoan_the_gioi': 4,
    },
    'doanh_nghiep': {
        'hoat_dong_kinh_doanh': 8,
    }
}

cafef_count_dict = {
    'trong_nuoc': {
        'kinh_te_vi_mo': 4,
        'chung_khoan': 4,
    },
    'quoc_te': {
        'tai_chinh_quoc_te': 6,
    },
    'doanh_nghiep': {
        'doanh_nghiep': 6,
    }
}

vietnambiz_count_dict = {
    'trong_nuoc': {
        'chung_khoan': 4,
        'nha_dat': 4,
    },
    'quoc_te': {
        'tai_chinh_chung_khoan': 6
    },
    'doanh_nghiep': {
        'ket_qua_kinh_doanh': 6
    }
}

In [5]:
full_news_df_dict = {}

# Khởi tạo dictionary với list rỗng cho mỗi news_type
for news_type in ['trong_nuoc', 'quoc_te', 'doanh_nghiep']:
    full_news_df_dict[news_type] = []

# Lấy tin từ VietStock
for news_type, url_dict in vietstock_rss_url_dict.items():
    for category_name, url in url_dict.items():
        max_entries = vietstock_count_dict[news_type][category_name]
        feed = feedparser.parse(url)
        
        # Lấy các bài mới nhất theo max_entries
        for entry in feed.entries[:max_entries]:
            # Get both content and image URL
            content, image_url = get_article_vietstock(entry['id'])
            
            # Lấy thời gian đăng bài từ RSS feed
            published_time = ""
            if hasattr(entry, 'published') and entry.published:
                published_time = entry.published
            
            full_news_df_dict[news_type].append({
                'source': 'VietStock',
                'title': entry['title'], 
                'content': summary_weekly_article(fast_model_dict, content, news_type),
                'image_url': image_url,
                'article_url': entry['id'],
                'published_time': published_time,
            })
            time.sleep(3)
        print(f"Thành công lấy tin từ VietStock: {category_name} ({url})")

# Lấy tin từ CafeF
for news_type, url_dict in cafef_rss_url_dict.items():
    for category_name, url in url_dict.items():
        max_entries = cafef_count_dict[news_type][category_name]
        # Lấy danh sách bài viết từ trang danh mục (tạo feed giả)
        feed_entries = get_cafef_articles_list(url, max_entries)
        
        # Lấy các bài mới nhất theo max_entries
        for entry in feed_entries[:max_entries]:
            # Get both content and image URL (giống VietStock)
            content, image_url = get_article_cafef(entry['id'])
            
            # Lấy thời gian đăng bài từ bài viết
            published_time = get_cafef_published_time(entry['id'])
            
            full_news_df_dict[news_type].append({
                'source': 'CafeF',
                'title': entry['title'], 
                'content': summary_weekly_article(fast_model_dict, content, news_type),
                'image_url': image_url,
                'article_url': entry['id'],
                'published_time': published_time,
            })
            time.sleep(3)
        print(f"Thành công lấy tin từ CafeF: {category_name} ({url})")

# Lấy tin từ Vietnambiz 
for news_type, url_dict in vietnambiz_rss_url_dict.items():
    for category_name, url in url_dict.items():
        max_entries = vietnambiz_count_dict[news_type][category_name]
        # Lấy danh sách bài viết từ trang danh mục (tạo feed giả)
        feed_entries = get_vietnambiz_articles_list(url, max_entries)
        
        # Lấy các bài mới nhất theo max_entries
        for entry in feed_entries[:max_entries]:
            # Get both content and image URL (giống VietStock)
            content, image_url = get_article_vietnambiz(entry['id'])
            
            # Lấy thời gian đăng bài từ bài viết
            published_time = get_vietnambiz_published_time(entry['id'])
            
            full_news_df_dict[news_type].append({
                'source': 'Vietnambiz',
                'title': entry['title'], 
                'content': summary_weekly_article(fast_model_dict, content, news_type),
                'image_url': image_url,
                'article_url': entry['id'],
                'published_time': published_time,
            })
            time.sleep(3)
        print(f"Thành công lấy tin từ Vietnambiz: {category_name} ({url})")

Thành công lấy tin từ VietStock: vi_mo (https://vietstock.vn/761/kinh-te/vi-mo.rss)
Thành công lấy tin từ VietStock: tai_chinh_quoc_te (https://vietstock.vn/772/the-gioi/tai-chinh-quoc-te.rss)
Thành công lấy tin từ VietStock: chung_khoan_the_gioi (https://vietstock.vn/773/the-gioi/chung-khoan-the-gioi.rss)
Thành công lấy tin từ VietStock: hoat_dong_kinh_doanh (https://vietstock.vn/737/doanh-nghiep/hoat-dong-kinh-doanh.rss)
Thành công lấy tin từ CafeF: kinh_te_vi_mo (https://cafef.vn/vi-mo-dau-tu.chn)
Thành công lấy tin từ CafeF: chung_khoan (https://cafef.vn/thi-truong-chung-khoan.chn)
Thành công lấy tin từ CafeF: tai_chinh_quoc_te (https://cafef.vn/tai-chinh-quoc-te.chn)
Thành công lấy tin từ CafeF: doanh_nghiep (https://cafef.vn/doanh-nghiep.chn)
Thành công lấy tin từ Vietnambiz: chung_khoan (https://vietnambiz.vn/chung-khoan/thi-truong.htm)
Thành công lấy tin từ Vietnambiz: nha_dat (https://vietnambiz.vn/nha-dat/thi-truong.htm)
Thành công lấy tin từ Vietnambiz: tai_chinh_chung_khoan

In [6]:
# Chuyển đổi dictionary thành pandas DataFrame
all_news_list = []
for news_type, articles in full_news_df_dict.items():
    for article in articles:
        # Thêm cột news_type vào mỗi article
        article_with_type = article.copy()
        article_with_type['news_type'] = news_type
        all_news_list.append(article_with_type)

# Tạo DataFrame
news_df = pd.DataFrame(all_news_list)
news_df['published_time'] = news_df['published_time'].apply(convert_published_time)
news_df = news_df[['news_type'] + [col for col in news_df.columns if col != 'news_type']]

# Thêm các cột cần thiết
news_df['word_count'] = news_df['content'].str.split().str.len()
news_df['impact'] = analyze_news_impact(standard_model_dict, news_df)
news_df['sectors'] = analyze_news_sectors(standard_model_dict, news_df)

In [7]:
%%capture
save_to_mssql(cts_engine, news_df, 'weekly_news')